<a href="https://colab.research.google.com/github/zhaocaiQ/Git-Tutorial/blob/master/modified_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##weekly_chart

In [ ]:
from datetime import datetime
date = datetime.today().strftime("%Y-%m-%d")

#DB keyword테이블에서 이번주 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = cursor.fetchall()
#이번주(월~일) 날짜 추출
cursor.execute("WITH RECURSIVE day AS (SELECT ADDDATE(curdate(), - WEEKDAY(curdate())) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < ADDDATE(curdate(), - WEEKDAY(curdate())+ 6)) SELECT date_format(DAY, '%Y-%m-%d') FROM day;")
day = cursor.fetchall()

#이번주 데이터가 없을 시 지난주 데이터 추출
if len(keywords) == 0:
  #DB keyword테이블에서 지난주 keyword별 언급량 합계 상위 5개의 keyword 추출
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= date_format(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate()), '%Y-%m-%d') AND DATE(c_date) <= date_format(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())), '%Y-%m-%d') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = cursor.fetchall()
  #지난주(월~일) 날짜 추출
  cursor.execute("WITH RECURSIVE day AS (SELECT DATE(curdate() - INTERVAL 1 WEEK - WEEKDAY(curdate())) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < DATE(curdate() - INTERVAL 1 WEEK + (6 - WEEKDAY(curdate())))) SELECT date_format(DAY, '%Y-%m-%d') FROM day;")
  day = cursor.fetchall()

weekly_chart = {}
#DB keyword테이블에서 추출한 keywords의 일별 weight 추출
for i in range(5):
  weight = [0] * 7
  no = 0
  for d in day:
    if d[0] == date:
      break
    else:
      #일별 추출한 keywords의 weight 추출
      cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) = '{d[0]}' AND keyword = '{keywords[i][0]}'")
      daily_weight = list(cursor.fetchall())
      if daily_weight != []:
        weight[no] = daily_weight[0][0]
      no += 1
  weekly_chart[keywords[i][0]] = weight
print(weekly_chart)

##monthly_chart

In [ ]:
from datetime import datetime
date = datetime.today().strftime("%Y-%m-%d")

#DB keyword테이블에서 이번달 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = cursor.fetchall()
#이번달(1일~말일) 날짜 추출
cursor.execute("WITH RECURSIVE day AS (SELECT DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < LAST_DAY(NOW()))SELECT date_format(DAY, '%Y-%m-%d') FROM day")
day = cursor.fetchall()

#이번달 데이터가 없을 시 지난달 데이터 추출
if len(keywords) == 0:
  #DB keyword테이블에서 지난달 keyword별 언급량 합계 상위 5개의 keyword 추출
  cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))) AND DATE(c_date) <= ADDDATE(date_add(curdate(), interval -1 day), - WEEKDAY(date_add(curdate(), interval -1 day))+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = cursor.fetchall()
  #지난달(1일~말일) 날짜 추출
  cursor.execute("WITH RECURSIVE day AS (SELECT DATE_SUB((curdate() - INTERVAL 1 MONTH), INTERVAL (DAY(curdate())-1) DAY) AS DAY UNION ALL SELECT DATE_ADD(DAY, INTERVAL 1 DAY) FROM day WHERE DAY < LAST_DAY((curdate() - INTERVAL 1 MONTH)))SELECT date_format(DAY, '%Y-%m-%d') FROM day;")
  day = cursor.fetchall()

monthly_chart = {}

#DB keyword테이블에서 추출한 keywords의 일별 weight 추출
for i in range(5):
  weight = [0] * int(day[-1][0][-2:])
  for d in day:
    if d[0] == date:
      break
    else:
      #일별 추출한 keywords의 weight 추출
      cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) = '{d[0]}' AND keyword = '{keywords[i][0]}'")
      daily_weight = list(cursor.fetchall())
      if daily_weight != []:
        weight[int(d[0][-2:])-1] = daily_weight[0][0]
  monthly_chart[keywords[i][0]] = weight
print(weekly_chart)

##monthly_youtube

In [ ]:
#DB keyword테이블에서 이번달 keyword별 언급량 합계 상위 5개의 keyword 추출
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
if len(keywords) != 0:
  #DB youtube테이블에서 추출한 keywords의 키워드, 조회수, 좋아요 수, 댓글 수, 해시태그 모두 추출
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube  WHERE SUBSTR(keyword_id, 1, 8) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND SUBSTR(keyword_id, 1, 8) <= LAST_DAY(NOW()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall()
else:
  #이번달 정보 없을 시 지난주 정보 출력
  cursor.execute("SELECT keyword FROM keyword WHERE date_format(c_date, '%Y-%m') = date_format((curdate() - INTERVAL 1 MONTH), '%Y-%m') GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
  keywords = list(cursor.fetchall()) 
  cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube  WHERE SUBSTR(keyword_id, 1, 5) = date_format((curdate() - INTERVAL 1 MONTH), '%y-%m') GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
  youtube = cursor.fetchall() 

#youtube테이블에서 추출한 정보 keyword별로 합치기
monthly_youtube = {}
for i in keywords:
  monthly_youtube[i[0]] = [0, 0, 0, '']
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for data in youtube:
        if word[0] in data[0]:
            monthly_youtube[word[0]][0] += data[1]
            monthly_youtube[word[0]][1] += data[2]
            monthly_youtube[word[0]][2] += data[3]
            monthly_youtube[word[0]][3] += data[4]
    no += 1
print(monthly_youtube)